In [11]:
import SNPLIB
import math
import numpy as np
import numpy.linalg as npl
import matplotlib.pyplot as plt
from numpy import random
from scipy import stats

In [12]:
Fst = 0.1
num_generations = 200
effective_sample_size = math.floor(num_generations/2/(1-math.exp(-Fst)))
num_snps = 100000
num_causal_snps = 10000
num_samples = 4000
num_traits = 10000
num_pairs = num_traits//2

In [13]:
aaf = 0.05+0.9*np.random.rand(num_snps)
pop_af = SNPLIB.UpdateAf(aaf,2,num_generations,effective_sample_size)
pop = np.zeros((num_samples,2),dtype='double',order='F')
pop[:,0] = np.sort(np.random.beta(0.1,0.1,num_samples))
pop[:,1] = 1.0-pop[:,0]
af = pop@pop_af

In [14]:
obj = SNPLIB.SNPLIB()
obj.GenerateIndividuals(af)
geno_d = obj.UnpackGeno()*random.randn(1,num_snps)

In [ ]:
"""
Same additive effects
Same direction
"""
true_genetic_corr = np.zeros(num_pairs)
true_env_corr = np.zeros(num_pairs)
sim_traits = np.zeros((num_samples, num_traits), dtype='double', order='F')
for i in range(num_pairs):
    all_ind = np.arange(num_snps)
    additive_snp = random.choice(all_ind,replace=False)
    all_ind = np.setdiff1d(all_ind,additive_snp)
    snp_ind1 = random.choice(all_ind,size=(num_causal_snps,1),replace=False).squeeze()
    all_ind = np.setdiff1d(all_ind,snp_ind1)
    num_shared_snps = random.randint(num_causal_snps)
    snp_ind2 = random.choice(snp_ind1, size=(num_shared_snps,1),replace=False).squeeze()
    snp_ind2 = np.concatenate((snp_ind2, random.choice(all_ind, size=(num_causal_snps-num_shared_snps,1),replace=False).squeeze()))
    rho = random.rand(1)*2-1
    env_effects = random.multivariate_normal([0.,0.],[[1,rho],[rho,1]],num_samples)
    effects_1 = np.zeros((num_samples,3),dtype='double',order='F')
    effects_1[:,0] = geno_d[:,additive_snp]
    effects_1[:,1] = np.sum(geno_d[:,snp_ind1], axis=1)
    effects_1[:,2] = env_effects[:,0]
    effects_1 = stats.zscore(effects_1,axis=0,ddof=1)
    effects_1,_ = npl.qr(effects_1)
    effects_1 *= math.sqrt(num_samples-1)
    effects_1[:,0] *= math.sqrt(0.02)
    effects_1[:,1] *= 0.7
    effects_1[:,2] *= 0.7
    sim_traits[:,2*i] = np.sum(effects_1, axis=1);
    effects_2 = np.zeros((num_samples,3),dtype='double',order='F')
    effects_2[:,0] = geno_d[:,additive_snp]
    effects_2[:,1] = np.sum(geno_d[:,snp_ind2], axis=1)
    effects_2[:,2] = env_effects[:,1]
    effects_2 = stats.zscore(effects_2,axis=0,ddof=1)
    effects_2,_ = npl.qr(effects_2)
    effects_2 *= math.sqrt(num_samples-1)
    effects_2[:,0] *= math.sqrt(0.02)
    effects_2[:,1] *= 0.7
    effects_2[:,2] *= 0.7
    sim_traits[:,2*i+1] = np.sum(effects_2, axis=1);
    true_genetic_corr[i],_ = stats.pearsonr(effects_2[:,1],effects_1[:,1])
    true_env_corr[i],_ = stats.pearsonr(effects_2[:,2],effects_1[:,2])
    

In [ ]:
"""
Same additive effects
Opposite direction
"""
true_genetic_corr = np.zeros(num_pairs)
true_env_corr = np.zeros(num_pairs)
sim_traits = np.zeros((num_samples, num_traits), dtype='double', order='F')
for i in range(num_pairs):
    all_ind = np.arange(num_snps)
    additive_snp = random.choice(all_ind,replace=False)
    all_ind = np.setdiff1d(all_ind,additive_snp)
    snp_ind1 = random.choice(all_ind,size=(num_causal_snps,1),replace=False).squeeze()
    all_ind = np.setdiff1d(all_ind,snp_ind1)
    num_shared_snps = random.randint(num_causal_snps)
    snp_ind2 = random.choice(snp_ind1, size=(num_shared_snps,1),replace=False).squeeze()
    snp_ind2 = np.concatenate((snp_ind2, random.choice(all_ind, size=(num_causal_snps-num_shared_snps,1),replace=False).squeeze()))
    rho = random.rand(1)*2-1
    env_effects = random.multivariate_normal([0.,0.],[[1,rho],[rho,1]],num_samples)
    effects_1 = np.zeros((num_samples,3),dtype='double',order='F')
    effects_1[:,0] = geno_d[:,additive_snp]
    effects_1[:,1] = np.sum(geno_d[:,snp_ind1], axis=1)
    effects_1[:,2] = env_effects[:,0]
    effects_1 = stats.zscore(effects_1,axis=0,ddof=1)
    effects_1,_ = npl.qr(effects_1)
    effects_1 *= math.sqrt(num_samples-1)
    effects_1[:,0] *= math.sqrt(0.02)
    effects_1[:,1] *= 0.7
    effects_1[:,2] *= 0.7
    sim_traits[:,2*i] = np.sum(effects_1, axis=1);
    effects_2 = np.zeros((num_samples,3),dtype='double',order='F')
    effects_2[:,0] = -geno_d[:,additive_snp]
    effects_2[:,1] = np.sum(geno_d[:,snp_ind2], axis=1)
    effects_2[:,2] = env_effects[:,1]
    effects_2 = stats.zscore(effects_2,axis=0,ddof=1)
    effects_2,_ = npl.qr(effects_2)
    effects_2 *= math.sqrt(num_samples-1)
    effects_2[:,0] *= math.sqrt(0.02)
    effects_2[:,1] *= 0.7
    effects_2[:,2] *= 0.7
    sim_traits[:,2*i+1] = np.sum(effects_2, axis=1);
    true_genetic_corr[i],_ = stats.pearsonr(effects_2[:,1],effects_1[:,1])
    true_env_corr[i],_ = stats.pearsonr(effects_2[:,2],effects_1[:,2])

In [ ]:
"""
Difference additive effects
"""
true_genetic_corr = np.zeros(num_pairs)
true_env_corr = np.zeros(num_pairs)
sim_traits = np.zeros((num_samples, num_traits), dtype='double', order='F')
for i in range(num_pairs):
    all_ind = np.arange(num_snps)
    additive_snp = random.choice(all_ind,replace=False)
    all_ind = np.setdiff1d(all_ind,additive_snp)
    snp_ind1 = random.choice(all_ind,size=(num_causal_snps,1),replace=False).squeeze()
    all_ind = np.setdiff1d(all_ind,snp_ind1)
    num_shared_snps = random.randint(num_causal_snps)
    snp_ind2 = random.choice(snp_ind1, size=(num_shared_snps,1),replace=False).squeeze()
    snp_ind2 = np.concatenate((snp_ind2, random.choice(all_ind, size=(num_causal_snps-num_shared_snps,1),replace=False).squeeze()))
    rho = random.rand(1)*2-1
    env_effects = random.multivariate_normal([0.,0.],[[1,rho],[rho,1]],num_samples)
    effects_1 = np.zeros((num_samples,3),dtype='double',order='F')
    effects_1[:,0] = geno_d[:,additive_snp]
    effects_1[:,1] = np.sum(geno_d[:,snp_ind1], axis=1)
    effects_1[:,2] = env_effects[:,0]
    effects_1 = stats.zscore(effects_1,axis=0,ddof=1)
    effects_1,_ = npl.qr(effects_1)
    effects_1 *= math.sqrt(num_samples-1)
    effects_1[:,0] *= math.sqrt(0.02)
    effects_1[:,1] *= 0.7
    effects_1[:,2] *= 0.7
    sim_traits[:,2*i] = np.sum(effects_1, axis=1);
    effects_2 = np.zeros((num_samples,3),dtype='double',order='F')
    effects_2[:,0] = geno_d[:,additive_snp]
    effects_2[:,1] = np.sum(geno_d[:,snp_ind2], axis=1)
    effects_2[:,2] = env_effects[:,1]
    effects_2 = stats.zscore(effects_2,axis=0,ddof=1)
    effects_2,_ = npl.qr(effects_2)
    effects_2 *= math.sqrt(num_samples-1)
    effects_2[:,0] *= math.sqrt(0.02)
    effects_2[:,1] *= 0.7
    effects_2[:,2] *= 0.7
    sim_traits[:,2*i+1] = np.sum(effects_2, axis=1);
    true_genetic_corr[i],_ = stats.pearsonr(effects_2[:,1],effects_1[:,1])
    true_env_corr[i],_ = stats.pearsonr(effects_2[:,2],effects_1[:,2])

### plt.plot(additive_effect2,'.')
plt.show()

In [4]:
rho = random.rand(1)*2-1
env_effects = random.multivariate_normal([0.,0.],[[1,rho],[rho,1]],num_samples)

In [52]:
effects_1 = np.zeros((num_samples,3),dtype='double',order='F')
effects_1[:,0] = geno_d[:,additive_snp]
effects_1[:,1] = np.sum(geno_d[:,snp_ind1], axis=1)
effects_1[:,2] = env_effects[:,0]
effects_1 = stats.zscore(effects_1,axis=0,ddof=1)
effects_1,_ = npl.qr(effects_1)
effects_1 *= math.sqrt(num_samples-1)
effects_1[:,0] *= math.sqrt(0.02)
effects_1[:,1] *= 0.7
effects_1[:,2] *= 0.7

In [53]:
a = np.sum(effects_1,axis=1)

In [55]:
np.std(a,ddof=1)

0.9999999999999999

In [38]:
num_shared_snps

1794